In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [2]:
data = spark.read.csv('SMSSpamCollection',inferSchema=True,sep='\t')

In [3]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [4]:
data = data.withColumnRenamed('_c0', 'calss').withColumnRenamed('_c1', 'text')

In [5]:
data.show()

+-----+--------------------+
|calss|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [6]:
from pyspark.sql.functions import length

In [7]:
data = data.withColumn('length',length(data['text']))

In [8]:
data.show()

+-----+--------------------+------+
|calss|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [10]:
data.groupBy('calss').mean().show()

+-----+-----------------+
|calss|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [11]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, 
                                CountVectorizer, IDF, 
                                StringIndexer)

In [21]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='calss', outputCol='label')

In [14]:
from pyspark.ml.feature import VectorAssembler

In [22]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

In [16]:
from pyspark.ml.classification import NaiveBayes

In [23]:
nb = NaiveBayes()

In [18]:
from pyspark.ml import Pipeline

In [24]:
data_prep_pipe = Pipeline(stages=(ham_spam_to_numeric,tokenizer,
                                 stop_remove,count_vec,idf, clean_up))

In [25]:
cleaner = data_prep_pipe.fit(data)

In [26]:
clean_data = cleaner.transform(data)

In [27]:
clean_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|calss|                text|length|label|          token_text|          stop_token|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13458,[8,12,33,6...|(13458,[8,12,33,6...|(13459,[8,12,33,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13458,[0,26,308,...|(13458,[0,26,308,...|(13459,[0,26,308,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13458,[2,14,20,3...|(13458,[2,14,20,3...|(13459,[2,14,20,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13458,[0,73,84,1...|(13458,[0,73,8

In [28]:
clean_data = clean_data.select('label','features')

In [29]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
|  1.0|(13459,[11,57,62,...|
|  0.0|(13459,[11,55,108...|
|  0.0|(13459,[133,195,4...|
|  1.0|(13459,[1,50,124,...|
|  1.0|(13459,[0,1,14,29...|
|  0.0|(13459,[5,19,36,4...|
|  1.0|(13459,[9,18,40,9...|
|  1.0|(13459,[14,32,50,...|
|  0.0|(13459,[42,99,101...|
|  0.0|(13459,[567,1745,...|
|  1.0|(13459,[32,113,11...|
|  0.0|(13459,[86,224,47...|
|  0.0|(13459,[0,2,52,13...|
|  0.0|(13459,[0,77,107,...|
|  1.0|(13459,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [30]:
training, test = clean_data.randomSplit([0.7,0.3])

In [31]:
spam_detector = nb.fit(training)

In [32]:
data.printSchema()

root
 |-- calss: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [33]:
test_result = spam_detector.transform(test)

In [34]:
test_result.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13459,[0,1,2,14,...|[-607.98008035579...|[1.0,1.2698010517...|       0.0|
|  0.0|(13459,[0,1,2,44,...|[-1058.6508675970...|[1.0,6.9688811483...|       0.0|
|  0.0|(13459,[0,1,5,15,...|[-743.00712393808...|[1.0,8.6114468628...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-877.47734216327...|[1.0,2.4857592246...|       0.0|
|  0.0|(13459,[0,1,10,15...|[-539.17651266813...|[1.0,1.9794752913...|       0.0|
|  0.0|(13459,[0,1,12,34...|[-873.57067684311...|[1.0,8.1315433196...|       0.0|
|  0.0|(13459,[0,1,13,35...|[-448.98014587799...|[1.0,1.5572913309...|       0.0|
|  0.0|(13459,[0,1,18,20...|[-813.80380103223...|[1.0,1.0927577013...|       0.0|
|  0.0|(13459,[0,1,22,26...|[-362.40470227581...|[1.0,1.0470356055...|       0.0|
|  0.0|(13459,[0

In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [36]:
acc_eval = MulticlassClassificationEvaluator()

In [37]:
acc = acc_eval.evaluate(test_result)

In [38]:
print('ACC of NB Model')
print(acc)

ACC of NB Model
0.9274684637461292
